In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import read_classification_dataset, features_histograms_mean_std
# import sklearn
from sklearnex import patch_sklearn
from sklearn.calibration import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

from kneed import KneeLocator

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
model_2 = Pipeline([
      ('Scaler', StandardScaler()),
      ('Imputer', KNNImputer()),
      ('PCA', PCA(n_components=18)),
      ('KNN', KNeighborsClassifier())
    ])

param_grid_2 = [
    {
        "Imputer": [KNNImputer(weights='distance')],
        "Imputer__n_neighbors":[1,3,7,21],
        "KNN__n_neighbors":[1,3,7,21],
        "KNN": [KNeighborsClassifier(weights="distance", n_jobs=-1)]
    },
]
grid_2 = GridSearchCV(model_2, param_grid_2, scoring="f1_weighted")
train, target, test = read_classification_dataset(2)
X, y = train.values, target.values.flatten()
grid_2.fit(X, y)
print(grid_2.best_score_)
print(grid_2.best_params_)
best_model_2 = grid_2.best_estimator_


c:\Users\Morri\Documents\Notebooks\ML\Project
0.8278571428571428
{'Imputer': KNNImputer(weights='distance'), 'Imputer__n_neighbors': 1, 'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'), 'KNN__n_neighbors': 7}


In [16]:
res = grid_2.cv_results_
param_scores = list((zip(res["rank_test_score"],res["mean_test_score"],res["params"], )))
param_scores.sort(key=lambda x: x[0])
param_scores

[(1,
  0.8278571428571428,
  {'Imputer': KNNImputer(weights='distance'),
   'Imputer__n_neighbors': 1,
   'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'),
   'KNN__n_neighbors': 7}),
 (2,
  0.8158571428571427,
  {'Imputer': KNNImputer(weights='distance'),
   'Imputer__n_neighbors': 21,
   'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'),
   'KNN__n_neighbors': 7}),
 (3,
  0.7997142857142857,
  {'Imputer': KNNImputer(weights='distance'),
   'Imputer__n_neighbors': 1,
   'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'),
   'KNN__n_neighbors': 3}),
 (4,
  0.7900476190476191,
  {'Imputer': KNNImputer(weights='distance'),
   'Imputer__n_neighbors': 7,
   'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'),
   'KNN__n_neighbors': 3}),
 (5,
  0.7877142857142856,
  {'Imputer': KNNImputer(weights='distance'),
   'Imputer__n_neighbors': 3,
   'KNN': KNeighborsClassifier(n_jobs=-1, weights='distance'),
   'KNN__n_neighbors': 3}),
 (6,
  0.7862380952380952,
  